In [1]:
from tensorflow import keras
import tensorflow as tf
# tf.compat.v1.get_default_graph
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
from pathlib import Path
from IPython import get_ipython
import cv2 # CV2 for image manipulation
%matplotlib inline

In [2]:
# local Notebook
label_description_path=r'/root/kaggle/label_descriptions.json'
sample_submission_path=r'/root/kaggle/sample_submission.csv'
train_path='/root/kaggle/train.csv'
DATA_DIR = Path('/root/kaggle/')

In [3]:
%%time
image_df = pd.read_csv(train_path)[:50000]
image_df.drop(['EncodedPixels', 'Height', 'Width'], axis=1,inplace=True)

CPU times: user 8.96 s, sys: 1.1 s, total: 10.1 s
Wall time: 10.1 s


In [4]:
def path_creater(parm):
    return f'{DATA_DIR}/train/{parm["ImageId"]}.jpg'

def geather_attribute_1hot(pram):
    c_a=[]
    c_a.append(pram['ClassId'])
    if str(pram['AttributesIds']).split(',')[0] != 'nan':
#         print(str(pram['AttributesIds']).split(',')) [int(x)+46 for x in str(pram['AttributesIds']).split(',')]
        c_a.extend([int(x)+46 for x in str(pram['AttributesIds']).split(',')])
    
#     creating_one_hot
    targets = np.array(list(c_a), dtype = np.int32)
    nb_classes = 46+341+46
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return np.asarray(res.reshape(list(targets.shape)+[nb_classes]).sum(axis=0))

In [5]:
# image_df.columns
image_df.tail()

,ImageId,ClassId,AttributesIds
49995,2645d84a17aaad6df76fde13d68e2458,17,NaN
49996,2645d84a17aaad6df76fde13d68e2458,24,NaN
49997,2645d84a17aaad6df76fde13d68e2458,23,NaN
49998,2645d84a17aaad6df76fde13d68e2458,5,"136,138,145,148,289,311,317"
49999,2645d84a17aaad6df76fde13d68e2458,0,"115,136,142,147,225,295,316,326"


In [6]:
%%time
image_df['ImageIdPath']=image_df.apply(path_creater, axis=1)
image_df['OneHot_class_attr']=image_df.apply(geather_attribute_1hot, axis=1)
image_df.drop([ 'ClassId','AttributesIds'], axis=1,inplace=True)

CPU times: user 8.75 s, sys: 62.7 ms, total: 8.81 s
Wall time: 8.81 s


In [13]:
IMG_SIZE = 400 #224
CHANNELS = 3 
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [14]:
BATCH_SIZE = 10 
# AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 500
def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    # Parse and preprocess observations in parallel
#     dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(parse_function, num_parallel_calls=2)
    
    
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
#     dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.prefetch(buffer_size=1)
    
    
    return dataset

In [15]:
# Shaflling data 
sufful_img_df=image_df.reindex(np.random.permutation(image_df.index))
# sufful_img_df

#Spliting data Train and validation
def split_vals(a,n): return a[:n], a[n:]
n_valid = 300 #int(len(sufful_img_df)/100*1)
n_trn = len(sufful_img_df)-n_valid
image_df, valid_df = split_vals(sufful_img_df, n_trn)
print('Traning and Validation dataset shapes!')
print(image_df.shape)
print(valid_df.shape)

Traning and Validation dataset shapes!
(49900, 3)
(100, 3)


In [16]:
X=image_df['ImageIdPath'].tolist()
y=np.array(image_df['OneHot_class_attr'].values.tolist())

X_train, X_test, y_train_bin, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

train_ds = create_dataset(X_train, y_train_bin)

In [17]:
# # Shaflling data 
# sufful_img_df=image_df.reindex(np.random.permutation(image_df.index))
# # sufful_img_df

# #Spliting data Train and validation
# def split_vals(a,n): return a[:n], a[n:]
# n_valid = int(len(sufful_img_df)/100*10)
# n_trn = len(sufful_img_df)-n_valid
# image_df, valid_df = split_vals(sufful_img_df, n_trn)
# print('Traning and Validation dataset shapes!')
# print(image_df.shape)
# print(valid_df.shape)

In [18]:
#prepration of train dataset
# X_train=image_df['ImageIdPath'].tolist()
# y_train_bin=np.array(image_df['OneHot_class_attr'].values.tolist())

In [19]:
# def _resize_image(image_path):
#     IMAGE_SIZE=256
#     img = cv2.imread(image_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
#     return img

In [20]:
# img_id='ffffbf7014a9e408bfbb81a75bc70638'
# img_path=f'{DATA_DIR}/train/{img_id}.jpg'
# _resize_image(img_path)

In [21]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(433, activation='sigmoid'))

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 396, 396, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 198, 198, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 198, 198, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 194, 194, 32)      12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 97, 97, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 97, 97, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 93, 64)        5

In [23]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
X_valid=valid_df['ImageIdPath'].tolist()
y_valid_bin=np.array(valid_df['OneHot_class_attr'].values.tolist())

In [ ]:
# model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)
model.fit(train_ds, epochs=10, validation_data=create_dataset(X_valid, y_valid_bin), batch_size=64)

Epoch 1/10
1256/4491 [=======>......................] - ETA: 10:45 - loss: 0.0499 - accuracy: 0.0480

## Evaluation

In [ ]:
loss,accuracy=model.evaluate(create_dataset(X_test, y_test),verbose=2)
print(f'Loss of our Model: {loss} and Model Accuracy: {accuracy}')

## History of model traning and ploting

In [ ]:
model.history.history.keys()

In [ ]:
# Displaying traning and validation loss
model_history_dict=model.history.history
train_loss=model_history_dict['loss']
val_loss=model_history_dict['val_loss']
epochs=range(1,len(train_loss) +1)
plt.plot(epochs,train_loss,'b',color='r',label='Training Loss')
plt.plot(epochs,val_loss,'b',color='g',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.clf()

In [ ]:
# Displaying traning and validation Accuracy
train_acc=model_history_dict['acc']
val_acc=model_history_dict['val_acc']
epochs=range(1,len(train_acc) +1)
plt.plot(epochs,train_acc,'b',color='r',label='Training Accuracy')
plt.plot(epochs,val_acc,'b',color='g',label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.clf()

In [ ]:
img_id='ffffbf7014a9e408bfbb81a75bc70638'
img_path=fr'{DATA_DIR}/train/{img_id}.jpg'
img = image.load_img(img_path,target_size=(400,400,3))
img = image.img_to_array(img)
img = img/255

In [ ]:
# classes = np.array(train.columns[2:])
proba = model.predict(img.reshape(1,400,400,3))
print(proba)
top_3 = np.argsort(proba[0])[:-4:-1]
print(top_3)
# for i in range(3):
#     print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
# plt.imshow(img)